# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [3]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [4]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [5]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [6]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [7]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [8]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [9]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [10]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [11]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [13]:
df1 = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot.T, 'euclidean'))),
                         index=cust_prod_pivot.columns, columns=cust_prod_pivot.columns)

df1.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.787373,0.796275,0.800742,0.802944,0.787945,0.791036,0.797964,0.805604,0.798353,...,0.799702,0.800574,0.795979,0.801046,0.800827,0.810783,0.790402,0.799438,0.794277,0.793504
"Appetizer - Mini Egg Roll, Shrimp",0.787373,1.000000,0.785990,0.789820,0.791309,0.781370,0.780892,0.793178,0.792411,0.782163,...,0.786352,0.785219,0.786083,0.791107,0.790730,0.789365,0.780792,0.789921,0.785161,0.784089
Appetizer - Mushroom Tart,0.796275,0.785990,1.000000,0.799139,0.795258,0.789820,0.789775,0.796437,0.800013,0.788451,...,0.792815,0.793292,0.794953,0.806723,0.804312,0.801090,0.790485,0.793792,0.792953,0.789206
Appetizer - Sausage Rolls,0.800742,0.789820,0.799139,1.000000,0.799588,0.794285,0.789574,0.798722,0.797260,0.791697,...,0.795991,0.798940,0.798392,0.810396,0.801247,0.798936,0.795023,0.798128,0.793726,0.792023
Apricots - Dried,0.802944,0.791309,0.795258,0.799588,1.000000,0.795519,0.789677,0.801077,0.800483,0.796009,...,0.794726,0.794783,0.797001,0.806803,0.801652,0.799955,0.794699,0.797166,0.797490,0.793691


### Step 2: Get the products purchased for a specific customer of your choice.

In [70]:
df2 = data.loc[data["CustomerID"]==33]["ProductName"]
display(df2.head())

527      Beer - Original Organic Lager
1017              Lamb - Pieces, Diced
3012    French Pastry - Mini Chocolate
3476               Beer - Rickards Red
3795                           Mangoes
Name: ProductName, dtype: object

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [71]:
df3 = [df1[e].sort_values(ascending=False)[1:6].index for e in df2]

df3_flatten = [i for e in df3 for i in e if not i in df2]

print(len(df3_flatten))

# No me sale ninguno repetido nunca... No sé si estoy haciendo algo mal

315


### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [53]:
lst = [[e,df3_flatten.count(e)] for e in set(df3_flatten)]

sorted(lst, key=lambda x: -x[1])[:5]

[['Water - Green Tea Refresher', 45],
 ['Bread - Hot Dog Buns', 27],
 ['Steam Pan - Half Size Deep', 24],
 ['Cheese - Cambozola', 15],
 ['Sauce - Hollandaise', 13]]

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [57]:
ids = pd.unique(data["CustomerID"])
recom = []

def create_recommendations(c_id,distance):
    step2 = data.loc[data["CustomerID"]==c_id]["ProductName"]
    step3 = [distance[e].sort_values(ascending=False)[1:6].index for e in step2]
    step3_flatten = [i for e in step3 for i in e if not i in step2]
    lst_aux = [[e,step3_flatten.count(e)] for e in set(step3_flatten)]
    return [e[0] for e in sorted(lst_aux, key=lambda x: -x[1])[:5]]

for e in ids:
    recom.append(create_recommendations(e, df1))
    
dic = {
    "CustomerID": ids,
    "Recommendations": recom
}

print(dic)

{'CustomerID': array([61288, 77352, 40094, 23548, 78981, 83106, 11253, 35107, 15088,
       26031, 86595, 46715, 91013, 69931, 19753,  9546, 10224, 72846,
       28976,  3267, 19260, 76530, 30196, 48818, 15166, 11443, 79060,
       82597, 62411, 42611, 45420, 83495, 47122, 30815, 53198, 18591,
       62034,  1428,  2187, 55162, 22345, 81876, 48641, 81034, 29681,
       87947, 28867, 69967, 94420, 83158, 84364, 44742,  7772, 47303,
       75791, 77756, 97324, 13912, 52700, 21734,  6817, 21095, 28875,
       89588,  1034, 54827, 29287, 11102, 52155, 50748, 88552, 80938,
       60481, 92168, 66232, 64348, 37510, 44408, 89254,   477, 51483,
       97201, 42157, 52091, 86065, 18027, 93882, 42303, 32010, 53997,
       63033, 76475, 98185, 84779, 82146, 54495,  3903, 79854, 25449,
       85496, 95017, 33759, 43652, 35873, 97769,   891, 85161, 25327,
       25624, 68416, 44090, 71595, 68336, 38497, 10016, 36751, 58421,
       45498, 50774, 79997, 45313, 55715, 40901,  1577,  3909, 18480,
     

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [58]:
df6 = pd.DataFrame(dic)

df6bis = pd.concat([df6["CustomerID"], df6["Recommendations"].apply(pd.Series)], axis = 1)\
.rename(columns={0: "Recom 1", 1: "Recom 2", 2: "Recom 3", 3: "Recom 4", 4: "Recom 5",})

display(df6bis.head())

,CustomerID,Recom 1,Recom 2,Recom 3,Recom 4,Recom 5
0,61288,Water - Green Tea Refresher,Bread - Hot Dog Buns,Raspberries - Fresh,Scallops 60/80 Iqf,Cheese - Cambozola
1,77352,Water - Green Tea Refresher,Bread - Hot Dog Buns,Bar - Granola Trail Mix Fruit Nut,Sauce - Hollandaise,Steam Pan - Half Size Deep
2,40094,Water - Green Tea Refresher,Bread - Hot Dog Buns,Cheese - Cambozola,Steam Pan - Half Size Deep,Sauce - Hollandaise
3,23548,Water - Green Tea Refresher,Bread - Hot Dog Buns,Bar - Granola Trail Mix Fruit Nut,Cheese - Cambozola,Steam Pan - Half Size Deep
4,78981,Water - Green Tea Refresher,Bread - Hot Dog Buns,Steam Pan - Half Size Deep,Wine - Alsace Gewurztraminer,Sauce - Hollandaise


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [59]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [89]:
cust_prod_pivot_bis = cust_prod_pivot.T.copy()

cust_prod_pivot_bis[0] = pd.Series(new_customer)

cust_prod_pivot_bis=cust_prod_pivot_bis.fillna(0)

cust_prod_pivot_bis=cust_prod_pivot_bis.T

distances = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot_bis, 'euclidean'))), 
                         index=cust_prod_pivot_bis.index, columns=cust_prod_pivot_bis.index)


similar_cust_bis = list(distances[0].sort_values(ascending=False)[1:].head().index)
sim_cust_prod_bis = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped_bis = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products_bis = grouped_bis.sort_values('Quantity', ascending=False).reset_index()
  
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot_bis.T[0]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recommendations[0] = list(recs['ProductName'])

display(recommendations[0])

['Wine - Redchard Merritt',
 'Cassis',
 'Bread - Calabrese Baguette',
 'Wine - Crozes Hermitage E.',
 'Thyme - Lemon, Fresh']

### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [91]:
df1_bis = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot_bis.T, 'euclidean'))),
                         index=cust_prod_pivot.columns, columns=cust_prod_pivot.columns)

df3_bis = [df1[e].sort_values(ascending=False)[1:6].index for e in new_customer.keys()]

df3_bis_flatten = [i for e in df3_bis for i in e if not i in new_customer.keys()]

lst_bis = [[e,df3_bis_flatten.count(e)] for e in set(df3_bis_flatten)]

sorted(lst_bis, key=lambda x: -x[1])[:5]

[['Water - Green Tea Refresher', 8],
 ['Wine - Alsace Gewurztraminer', 5],
 ['Bread - Hot Dog Buns', 4],
 ['Bar - Granola Trail Mix Fruit Nut', 3],
 ['Milk Powder', 2]]